# Setup inicial

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [0]:
# Configurações visuais
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="viridis", font_scale=1.1)
plt.rcParams['figure.figsize'] = (10, 6)

In [0]:
# Ingestão e conversão do df para pandas
df_silver = spark.table("projeto_final_silver.silver_panorama_eja")
df = df_silver.toPandas()

# Visão Geral do Dataset

In [0]:
df.head()

In [0]:
df.info()

# Criação de Métricas

## Percentual Não Alfabetizado

In [0]:
df['pct_nao_alf'] = (
    100 * df['populacao_nao_alfabetizada_censo_ibge_2022'] /
    df['populacao_total_municipio_15_anos_censo_ibge_2022']
)

In [0]:
df_regiao = df.groupby('regiao').apply(
    lambda x: (x['populacao_nao_alfabetizada_censo_ibge_2022'].sum() /
               x['populacao_total_municipio_15_anos_censo_ibge_2022'].sum()) * 100
).reset_index(name='pct_nao_alf_regiao')

df_regiao

## Intensidade de Matriculados no EJA

In [0]:
df['matriculas_eja_per_1k'] = (
    df['matriculas_eja_fundamental_medio'] /
    df['populacao_total_municipio_15_anos_censo_ibge_2022'] * 1000
)

## Proporção de Não Alfabetizados x Faixa Etária

In [0]:
faixas = [
    '15_19_anos_populacao_nao_alfabetizada',
    '20_24_anos_populacao_nao_alfabetizada',
    '25_34_anos_populacao_nao_alfabetizada',
    '35_44_anos_populacao_nao_alfabetizada',
    '45_54_anos_populacao_nao_alfabetizada',
    '55_64_anos_populacao_nao_alfabetizada',
    '65_anos_ou_mais_populacao_nao_alfabetizada'
]
for col in faixas:
    df[f'share_{col}'] = 100 * df[col] / df['populacao_nao_alfabetizada_censo_ibge_2022']

# Visualizações

## Distribuição de Taxas de Analfabestismo por Região

In [0]:
plt.figure(figsize=(10,6))
sns.boxplot(data=df, x='regiao', y='taxa_analfabetismo_censo_ibge_2022')
plt.title('Distribuição da Taxa de Analfabetismo por Região')
plt.ylabel('Taxa de analfabetismo (%)')
plt.xlabel('')
plt.show()

## Top 10 municípios com maior taxa

In [0]:
top10 = df.nlargest(10, 'taxa_analfabetismo_censo_ibge_2022')
fig = px.bar(top10, x='municipio', y='taxa_analfabetismo_censo_ibge_2022',
             color='regiao', text_auto='.2f',
             title='Top 10 Municípios com Maior Taxa de Analfabetismo')
fig.show()

## Relação entre taxa de analfabetismo e matrículas EJA

In [0]:
fig = px.scatter(df, x='matriculas_eja_per_1k', y='taxa_analfabetismo_censo_ibge_2022',
                 color='regiao', size='populacao_total_municipio_15_anos_censo_ibge_2022',
                 hover_name='municipio',
                 title='Taxa de Analfabetismo vs Matrículas EJA por Município')
fig.show()

Nota de Mary: Quanto maior a quantidade de matriculas no eja, menor na taxa de analfabetismo. Mas não sei se o gráfico ta se comportando bem

## Correlação entre as variávies numéricas

In [0]:
corr_cols = ['taxa_analfabetismo_censo_ibge_2022', 'pct_nao_alf', 'matriculas_eja_per_1k',
             'escolas_municipais_eja', 'escolas_estaduais_eja']
plt.figure(figsize=(8,6))
sns.heatmap(df[corr_cols].corr(), annot=True, cmap='Blues')
plt.title('Correlação entre Variáveis')
plt.show()


## Adesão ao PACTO EJA por municipio

In [0]:
adesao_count = df.groupby("ano_adesao").size().reset_index(name="total_municipios")
adesao_count = adesao_count.sort_values("ano_adesao")

In [0]:
# Cria lista de cores
cores = []
for val in adesao_count["ano_adesao"]:
    if val == 'não':
        cores.append("#ff7f0e")  # azul
    else:
        cores.append("#1f77b4")  # cinza intermediário

# Gráfico
plt.figure(figsize=(8,5))
ax = sns.barplot(data=adesao_count, x="ano_adesao", y="total_municipios", palette=cores)

# Adiciona valores no topo
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom', fontsize=10)

plt.title("Número de municípios por ano de adesão")
plt.xlabel("Ano de adesão")
plt.ylabel("Quantidade de municípios")
plt.show()

In [0]:
adesao_count_UF = df.groupby("ano_adesao").size().reset_index(name="UF")

In [0]:
import plotly.express as px

# Média da taxa de analfabetismo por região
taxa_regiao = df.groupby("regiao")["taxa_analfabetismo_censo_ibge_2022"].mean().reset_index()

# Identifica a região com maior taxa
maior_valor = taxa_regiao["taxa_analfabetismo_censo_ibge_2022"].max()
taxa_regiao["cor"] = taxa_regiao["taxa_analfabetismo_censo_ibge_2022"].apply(
    lambda x: "Maior" if x == maior_valor else "Normal"
)

# Define cores: azul para normal, laranja para maior
cores = {"Normal":"#1f77b4", "Maior":"#ff7f0e"}

# Gráfico
fig = px.bar(taxa_regiao, x="regiao", y="taxa_analfabetismo_censo_ibge_2022",
             text_auto='.2f', color="cor", color_discrete_map=cores,
             title="Taxa Média de Analfabetismo por Região")
fig.update_layout(showlegend=False)
fig.show()

In [0]:
%sql
select distinct(uf)
from projeto_final_silver.silver_panorama_eja